<a href="https://colab.research.google.com/github/shizoda/education/blob/main/machine_learning/text/Using_LLM_via_OpenRouter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 OpenRouter API 呼び出し演習

このノートブックは、OpenRouter を使ってさまざまな LLM (大規模言語モデル) の API を呼び出す練習をするための教材です。

解説を読み、Markdown セルに記載されている **サンプルコード（解答例）** を参考にしながら、 **Python コードセル（練習問題）** の `...` の部分を埋めて、コードを実行してください。

---
## 🌎 1. OpenRouter とは？

OpenRouter は、**「LLM のルーター（仲介・振り分け役）」** のようなサービスです。

通常、OpenAI のモデル (GPT) を使いたい場合は OpenAI の API キーが、Anthropic のモデル (Opus, Sonnet など) を使いたい場合は Anthropic の API キーが... というように、モデル提供会社ごとに契約とAPIキーが必要で、呼び出し方も少しずつ違います。

OpenRouter を使うと、以下の利点があります。

1.  **単一の API キー:** OpenRouter の API キーが 1 つあれば、OpenRouter が提携している数十種類以上のモデル（OpenAI, Google, Anthropic, Meta, Mistral, Qwen, DeepSeek など）をすべて呼び出せます。
2.  **共通の API 形式:** すべてのモデルを、OpenAI の API と**まったく同じ形式**で呼び出せます。
3.  **コスト管理:** 利用料金は OpenRouter にまとめて支払う（クレジットカード登録）だけで済み、モデルごとのコストもダッシュボードで一目でわかります。
4.  **安価・オープンなモデルへのアクセス:** GPT-4 のような高性能モデルだけでなく、`Llama ` や `Qwen`、`DeepSeek` といった、非常に安価（または無料枠がある）で高性能なオープンソースモデルにも簡単にアクセスできます。

この演習では、この「OpenRouter を OpenAI 互換 API として利用する」方法を学びます。

---
## 🔑 2. 準備: API キーとライブラリのインストール

API を呼び出すための準備をします。

### ステップ 2-1: OpenRouter の API キーを Colab に設定する

まず、OpenRouter の API キーを取得し、Colab のシークレット機能に保存します。

1.  [OpenRouter のサイト](https://openrouter.ai/keys) にアクセスし、アカウント登録・ログインします。
2.  （初回利用時）クレジットカード情報を登録し、利用上限額（例: $10）を設定します。
3.  「Keys」ページで「+ Create Key」を押し、キーを発行します。（`sk-or-...` で始まる文字列です）
4.  Colab の画面左側にある「🔑」（鍵マーク）アイコンをクリックします。
5.  「新しいシークレットを追加」を押します。
    * **名前 (name):** `OPENROUTER_API_KEY`
    * **値 (value):** 先ほどコピーした `sk-or-...` のキー
6.  「ノートブックへのアクセス」を **オン (有効)** にします。



### ステップ 2-2: 必要なライブラリのインストール

OpenRouter は OpenAI API と互換性があるため、**`openai` ライブラリ**をそのまま使います。

In [ ]:
# 練習 2-2: openai ライブラリをインストールします
# (ヒント: !pip install ...)

...

print("openai ライブラリのインストールが完了しました。")

---
## 🔌 3. クライアントの初期化

API を呼び出すための「クライアント」を設定します。

シークレットから API キーを読み込み、OpenAI ライブラリのクライアントを作成します。
このとき、`base_url` (APIの接続先) を OpenAI 公式ではなく、OpenRouter の URL (`https://openrouter.ai/api/v1`) に指定するのが**最重要ポイント**です。

### 📑 サンプルコード (クライアント初期化)

```python
from openai import OpenAI
from google.colab import userdata

# 1. Colab シークレットからキーを取得
my_openrouter_key = userdata.get("OPENROUTER_API_KEY")

# 2. クライアントを初期化
client = OpenAI(
  # 接続先を OpenRouter に指定 (重要)
  base_url="https://openrouter.ai/api/v1,
  # 取得した API キーを指定
  api_key=my_openrouter_key,
)

print("OpenRouter クライアントの初期化に成功しました。")

In [ ]:
from openai import OpenAI
from google.colab import userdata

# 練習 3-1: Colab シークレットから "OPENROUTER_API_KEY" という名前のキーを取得
my_openrouter_key = userdata.get("...") # 👈 ここにシークレット名を入力

# 練習 3-2: OpenAI クライアントを初期化
client = OpenAI(
  # 接続先 (base_url) を OpenRouter のエンドポイントに指定
  base_url= "..." , # 👈 ここに OpenRouter の URL を入力

  # API キー (api_key) に上で取得した変数を指定
  api_key= ...  , # 👈 ここに my_openrouter_key 変数を入力
)

---
## 📜 4. モデルタイプ1: Text Completion (単純なテキスト補完)

これは、LLM の API としては古い形式 (`/v1/completions`) ですが、「与えられた文章の**続き**をそのまま書く」というモデルの基本的な動作を理解するのに役立ちます。

* **API:** `client.completions.create()` を使います。
* **特徴:** 対話形式ではなく、単一の「プロンプト (prompt)」を与えます。
* **用途:** 指示に従わせる (Reasoning) というよりは、文章の続きを予測させる (Non-Reasoning) タスクに向いています。
* **モデル例:** `nousresearch/nous-hermes-llama-2-7b` (Free Tier), `openai/gpt-3.5-turbo-instruct`

### 📑 サンプルコード (Text Completion)

```python
# 1. Text Completion をサポートするモデル名を指定
# (Free Tier のモデルを使ってみます)
model_name = "nousresearch/nous-hermes-llama-2-7b"

# 2. 続きを書いてほしいプロンプト
prompt_text = "昔々あるところに、"

# 3. API を呼び出す
response = client.completions.create(
    model=model_name,
    prompt=prompt_text,
    max_tokens=50 # 生成する最大の長さ
)

# 4. 結果（続きの文章）を表示
print(f"--- {model_name} が生成した続き ---")
print(prompt_text + response.choices[0].text)

In [ ]:
# 練習 4-1: モデル名を指定 (例: "deepseek/deepseek-chat-v3-0324")
model_name = "..." # 👈 ここにモデル名を入力

# 練習 4-2: 続きを書いてほしいプロンプト
# (例: "昔々あるところに")
prompt_text = "..." # 👈 ここにプロンプトを入力

# 練習 4-3: client.completions.create() を呼び出す
# (ヒント: model, prompt, max_tokens が必要です)

# client.completions.create メソッド（メソッドとは関数みたいなもの）を実行し，
# 戻り値を response として受け取ってください
... = ... (
    model=model_name,
    prompt=prompt_text,
    max_tokens=30
)

# 練習 4-4: 結果（プロンプト + 続き）を表示
# (ヒント: 続きは response.choices[0].text に入っています)
generated_text = ... # 👈 ここに続きのテキストを取り出すコードを入力

print(f"--- {model_name} による補完結果 ---")
print(prompt_text + generated_text)

---
## 🧠 5. モデルタイプ2: Chat Completion (対話・指示実行)

現在、API 利用の主流となっている形式 (`/v1/chat/completions`) です。
単純な続きを書くだけでなく、役割を与えたり、複雑な指示 (Reasoning) に従わせたりすることができます。

* **API:** `client.chat.completions.create()` を使います。
* **特徴:** `messages` リストを使って、役割 (`role`) ごとに発言を渡します。
    * `system`: AI の役割や前提条件を指示します (例: 「あなたはPythonの専門家です」)。
    * `user`: ユーザーからの質問や指示です。
    * `assistant`: (オプション) AI の過去の応答例です。
* **モデル例:**
    * `qwen/qwen3-coder-flash` (コーディング向き・安価)
    * `meta-llama/llama-3.1-8b-instruct` (高性能・安価)

### 📑 サンプルコード (Chat Completion)

```python
# 1. Chat Completion モデル名を指定 (コーディングに強いモデル)
model_name = "deepseek/deepseek-coder-v2-lite"

# 2. 送信するメッセージのリストを作成
messages = [
    {"role": "system", "content": "あなたはPythonコーディングの専門家です。"},
    {"role": "user", "content": "Pythonで、リストの偶数だけを返す関数を書いてください。"}
]

# 3. API を呼び出す
response = client.chat.completions.create(
    model=model_name,
    messages=messages
)

# 4. 結果（回答）を表示
print(f"--- {model_name} からの回答 ---")
print(response.choices[0].message.content)

In [ ]:
# 練習 5-1: モデル名を指定 (例: "qwen/qwen-2-7b-instruct")
model_name = "..." # 👈 ここにチャットモデル名を入力

# 練習 5-2: メッセージリストを作成 (system と user)
messages = [
    {"role": "system", "content": "あなたは日本語で回答する、親切なAIアシスタントです。"},
    {"role": "user", "content": "OpenRouterの利点を2つ教えてください。"}
]

# 練習 5-3: client.chat.completions.create() を呼び出す
# (ヒント: model と messages が必要です)
... = ... (
    model= ... , # 👈 ここに model_name 変数を入力
    messages= ...  # 👈 ここに messages 変数を入力
)

# 練習 5-4: 結果（回答）を表示
# (ヒント: 回答は response.choices[0].message.content に入っています)
answer = ... # 👈 ここに回答を取り出すコードを入力

print(f"--- {model_name} からの回答 ---")
print(answer)

---
## 🎉 6. まとめ

お疲れ様でした！この演習では、OpenRouter を通じて2つの異なるタイプの LLM を呼び出す方法を学びました。

1.  **Text Completion (<code>completions.create</code>):**
    * プロンプトの「続き」を生成する。
2.  **Chat Completion (<code>chat.completions.create</code>):**
    * 現在の主流。`messages` リストで対話や指示 (Reasoning) を行う。